### Generates the Records_df and from that computes and exports Frequency_df
### Frequency_df is the only one directly used by the Pathapp

In [2]:
"""File to generate sample data for testing."""
from datetime import date, timedelta, datetime
import random
import math
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [2]:
from sample_data import type_list,value_sign,day,adequacy_list,choices,last_week_date, last_month_date,results_list, genotype_list

## Generates the numer of tests processed in a given date

In [3]:
### this function defines how busy each day of the week is
def busy_days(n:int):
    ans=100
    if n==6:
        ans=0
    return ans
    
    
    

In [4]:
### this function defines how busy each month is
def busy_months(n:int):
    ans=1
    if n ==12:
        ans= 0.8
    if n in [1,2,6,7]:
        ans=1.2
    return ans    
    
    

In [5]:
def number_of_tests(fecha): ### This takes a date and returns how many tests occur
    ans=int (busy_days(fecha.weekday())*busy_months(fecha.month)*np.random.uniform(0.95,1.05))
    return ans


## Generate a new row given a data

In [7]:
def make_row(fecha):############ This takesa a date and returns an entry in the Records_df
    type_of_test=random.choices(type_list,weights=[3,4])[0]
    adequacy=random.choices(adequacy_list,weights=[5,2,1])[0]
    result=random.choices(results_list,weights=[4,1,1,1,1])[0]
    gen_type=random.choice(genotype_list)
    cytology=random.choices(value_sign,weights=[4,1])[0]+"cytology"
    hystology="Not prescribed"
    if cytology=="Positivecytology":
        hystology=random.choices(value_sign,weights=[92,8])[0]+"hystology"
    
    new_row = pd.DataFrame([{'day':fecha.date(), 'type':type_of_test,'adequacy':adequacy, 'result':result,'genotype':gen_type,'cytology':cytology,'hystology':hystology}])
    return new_row

## Choose the date range 

In [8]:
starting_date = date.today() - timedelta(days = 366)
ending_date=date.today() +timedelta(days = 20)

## Generate Records_df

In [9]:
def generate_test_df():
    columns=['day', 'type', 'adequacy', 'result', 'genotype', 'cytology', 'hystology']
    records_df = pd.DataFrame(columns=columns)
    daterange = pd.date_range(starting_date, ending_date, freq='D')
    for item in daterange:
        N_of_test=number_of_tests(item)
        for patient in range(N_of_test):
            new_row=make_row(item)
            records_df = pd.concat([records_df, new_row], axis=0, ignore_index=True)
    return records_df

In [10]:
Records_df=generate_test_df()

### This is the dataframe that the app takes

In [17]:
Records_df.to_csv("Records.csv")

### GENERATE FREQUENCY DATA FRAME

In [18]:
def new_row_frequency(df,fecha):
    columnas= list(Records_df.columns)[1:]
    new_row_in={"day":fecha}
    df_fecha=df[df["day"]==fecha]
    new_row_in["All"]=df_fecha.shape[0]
    for columna in columnas:
        column_values=list(df_fecha[columna].unique())
        for tipo in column_values:
            temp_df=df_fecha[df_fecha[columna]==tipo]
            new_row_in[tipo]=temp_df.shape[0]
    types=list(df_fecha["type"].unique())
    adequacies=list(df_fecha["adequacy"].unique())
    cytologies=list(df_fecha["cytology"].unique())
    hystologies=list(df_fecha["hystology"].unique())
    for tipo in types:
        for ad in adequacies:
            temp_df=df_fecha[(df_fecha["type"]==tipo) & (df_fecha["adequacy"]==ad)]
            new_row_in[tipo+ad]=temp_df.shape[0]
        for cyt in cytologies:
            temp_df=df_fecha[(df_fecha["type"]==tipo) & (df_fecha["cytology"]==cyt)]
            new_row_in[tipo+cyt]=temp_df.shape[0]
        for hyst in hystologies:
            temp_df=df_fecha[(df_fecha["type"]==tipo) & (df_fecha["hystology"]==hyst)]
            new_row_in[tipo+hyst]=temp_df.shape[0]


        # new_row_in["weekday"]=week_day(fecha.weekday())
    
    return pd.DataFrame([new_row_in])

In [19]:
def make_frequency_df(df):
    fechas=df["day"].unique()
    freq_df=pd.DataFrame()
    for fecha in fechas:
        freq_df=pd.concat([freq_df,new_row_frequency(df,fecha)],axis=0, ignore_index=True)
    
    return freq_df

In [20]:
Frequency_df=make_frequency_df(Records_df)

In [17]:
#Frequency_df.to_csv('Frequency.csv')#

In [5]:
df_caribe=pd.read_excel("assets/datos-cliente.xlsx")

In [28]:
df_caribe.head(20)

,Informe No.,max(v.fechaanalisis),identificacioncliente,ubicacion,nombrepunto,prefijo,Muestra No.,pH,Al,MO,...,CICE,CE,P,Nitrato,Amonio,Fe,Mn,Cu,Zn,B
0,9,2021-02-04 05:00:00,Bloque 7A Camas 1-200,San Miguel,San Sebastian,SF,1,"5,4",0.32,"14,51",...,28.32,"0,84","78,67","147,4",51,"116,52","5,57","1,23","5,25","0,96"
1,9,2021-02-04 05:00:00,Bloque 2 Camas 1-296,San Miguel,San Sebastian,SF,2,"5,5",N.A.,"15,72",...,25,"0,86","40,39","182,6","53,78","113,44","4,06","1,05","3,28","1,2"
2,24,2021-02-15 05:00:00,Bloque 12C Cama 1 - 180,El Yaruno,Plantas Madres,SF,127,"8,5",N.A.,"16,82",...,79.1,"4,70","356,95","1534,3","28,15","88,55","31,83","3,90","9,28","1,81"
3,25,2021-02-15 05:00:00,Bloque 4,Gualanday,Valley,SF,128,"7,2",N.A.,"13,30",...,"38,19","0,95","172,40","391,0","20,28","56,85","3,92","2,88","5,48","1,59"
4,44,2021-02-22 05:00:00,Bloque 4 Semana 5 General - 05/02/2020,El Tablazo,Alhambra,SF,248,"6,9",N.A.,"7,96",...,"26,23","0,74","366,07","114,9","11,47","85,69","5,56","7,62","5,98","2,84"
5,44,2021-02-22 05:00:00,Bloque 2 Semana 5 General - 05/02/2020,El Tablazo,Alhambra,SF,249,"7,0",N.A.,"7,66",...,"25,31","0,88","366,32","51,2","14,22","110,97","6,44","15,76","7,18","3,05"
6,44,2021-02-22 05:00:00,Bloque 6 Semana 5 General - 05/02/2020,El Tablazo,Alhambra,SF,250,"6,9",N.A.,"11,57",...,"31,35","1,21","301,80","134,4","13,77","126,28","4,81","6,14","6,72","2,30"
7,57,2021-02-22 05:00:00,Bloque 4 Camas 1 - 176,San Miguel,San Sebastian,SF,251,"5,7",N.A.,"14,06",...,"14,53","1,20","19,99","278,6","23,32","101,29","3,82",N.D.,"2,02","0,60"
8,57,2021-02-22 05:00:00,Bloque 6 Camas 1 - 176,San Miguel,San Sebastian,SF,252,"5,6",N.A.,"12,55",...,"13,71","0,91","15,11","232,5","16,72","139,24","3,68","0,35","2,65","0,37"
9,47,2021-02-22 05:00:00,Bloque 11 Semana 5 Genera - 05-02-2020,El Tablazo,Alhambra,SF,253,"5,5",N.A.,"5,07",...,"4,37","0,07","22,45","5,9","14,03","140,56","11,27","4,87","1,06","0,39"


In [7]:
df_caribe.to_csv("datos_caribe.csv")

In [26]:
d

str